In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import os
from tqdm import tqdm
from sklearn import linear_model
import helper_functions as hf 
from sklearn import linear_model


repo_path = '/home/tudor/Documents/GitHub/building_resilience'
data_path = repo_path + '/data/top_to_bottom/data'

In [2]:
## read data files:
data_path = repo_path + '/data/top_to_bottom/data'

# country codes for countries written in the UN format
CCs = pd.read_csv(data_path + '/help_data/UN_country_code.csv')
CCs = CCs.dropna(axis = 0).reset_index(drop=True)

# country codes for countries written in the WB format
WBs = pd.read_csv(data_path + '/help_data/WB_country_code.csv')
WBs = WBs.dropna(axis = 0).reset_index(drop=True)

#read WB data
WB_data = pd.read_csv(data_path+'/WB_WDI_data/wb_data.csv')

#read UN data
UN_data = pd.read_csv(data_path+'/UN_data/WPP2019_TotalPopulationBySex.csv')

# #read Climate reanalysis data
# climate_data = pd.read_csv(data_path+'/climate_data/era5_reanalysis.csv')

# #read GCM climate prediction data:
# prediction_data = pd.read_csv(data_path+'/climate_data/future_final.csv')

# read CDD data from Cimate reanalysis data and GCM prediction data:
CDD = pd.read_csv(data_path+'/climate_data/countrywise_CDDs.csv')
CDD = CDD.drop(columns=['Unnamed: 0'])
CDD = CDD.replace(0, np.nan)
CDD = CDD.dropna().reset_index(drop=True)

In [3]:
UN_prediction = hf.choose_UN_scenario(data_path,UN_data,'Medium')
UN_prediction=hf.drop_unwanted_UN_years(UN_prediction)

Medium scenario population prediction data succesfully extracted in your variable!


In [13]:
# initialise datasets for electricity:

industrial = pd.read_csv('data/help_data/WB_country_code.csv').dropna(axis = 0).reset_index(drop=True)
residential = pd.read_csv('data/help_data/WB_country_code.csv').dropna(axis = 0).reset_index(drop=True)
commercial = pd.read_csv('data/help_data/WB_country_code.csv').dropna(axis = 0).reset_index(drop=True)

for i in range(15):

    year = i+2000
    WB_dataset = hf.get_year_of_WB_data(year,WB_data)
    iea_path = 'data/IEA_data/IEA_percentage_breakdown_'+str(hf.IEA_year(year))+'.csv'
    IEA_dataset = pd.read_csv(iea_path)
    IEA_dataset = IEA_dataset.drop(['split','new_Data'], axis=1)

    WB_IEA = hf.merge_WB_and_IEA(WB_dataset,IEA_dataset)
    ## WorldBank has inputed thier NaN values as '..', so we need to remove them
    WB_IEA = WB_IEA.replace(to_replace='..', value=np.nan)
    WB_IEA = WB_IEA.dropna(axis=0).reset_index(drop=True)


    industry_elec_list = []
    residential_elec_list = []
    commercial_elec_list = []

    for i in range(len(WB_IEA)):
        df = WB_IEA.iloc[i,:]
        ind_elec = float(df.iloc[3])*float(df.iloc[5])*float(df.iloc[2])*float(df.iloc[6])*0.2
        industry_elec_list.append(ind_elec)
        res_elec = float(df.iloc[3])*float(df.iloc[5])*float(df.iloc[2])*float(df.iloc[7])*0.2
        residential_elec_list.append(res_elec)
        com_elec = float(df.iloc[3])*float(df.iloc[5])*float(df.iloc[2])*float(df.iloc[8])*0.2
        commercial_elec_list.append(com_elec)


    WB_IEA['Cooling industry electricity (kWh)'] = industry_elec_list
    WB_IEA['Cooling residential electricity (kWh)'] = residential_elec_list 
    WB_IEA['Cooling commercial electricity (kWh)'] = commercial_elec_list

    df1 = WB_IEA[['Country Code','Cooling industry electricity (kWh)']]
    industrial = industrial.merge(df1, on= 'Country Code', how = 'left')
    industrial = industrial.rename(columns={"Cooling industry electricity (kWh)": str(year)})
    
    df2 = WB_IEA[['Country Code','Cooling residential electricity (kWh)']]
    residential = residential.merge(df2, on= 'Country Code', how = 'left')
    residential = residential.rename(columns={"Cooling residential electricity (kWh)": str(year)})


    df3 = WB_IEA[['Country Code','Cooling commercial electricity (kWh)']]
    commercial = commercial.merge(df3, on= 'Country Code', how = 'left')
    commercial = commercial.rename(columns={"Cooling commercial electricity (kWh)": str(year)})

industrial = industrial.dropna(thresh = 4).reset_index(drop=True)
residential = residential.dropna(thresh = 4).reset_index(drop=True)
commercial = commercial.dropna(thresh = 4).reset_index(drop=True)


WB data for the year 2000 was succesfully imported!
WB data for the year 2001 was succesfully imported!
WB data for the year 2002 was succesfully imported!
WB data for the year 2003 was succesfully imported!
WB data for the year 2004 was succesfully imported!
WB data for the year 2005 was succesfully imported!
WB data for the year 2006 was succesfully imported!
WB data for the year 2007 was succesfully imported!
WB data for the year 2008 was succesfully imported!
WB data for the year 2009 was succesfully imported!
WB data for the year 2010 was succesfully imported!
WB data for the year 2011 was succesfully imported!
WB data for the year 2012 was succesfully imported!
WB data for the year 2013 was succesfully imported!
WB data for the year 2014 was succesfully imported!


In [185]:
intersect_country = pd.read_csv('data/help_data/WB_country_code.csv')

cdd_count = []
pop_count = []
elec_count= []

for i in range(len(intersect_country)):
    code = intersect_country.iloc[i]['Country Code']
    
    if (len(CDD[CDD['Country Code'] == code]) == 1):
        cdd_count.append(1)
    else:
        cdd_count.append(np.nan)
        
    if (len(UN_prediction[UN_prediction['Country Code'] == code]) == 1):
        pop_count.append(1)
    else:
        pop_count.append(np.nan)
        
    if (len(industrial[industrial['Country Code'] == code]) == 1):
        elec_count.append(1)
    else:
        elec_count.append(np.nan)
        
intersect_country['CDD'] = cdd_count
intersect_country['pop'] = pop_count
intersect_country['elec']=elec_count


intersect_country = intersect_country.dropna().reset_index(drop=True)


In [188]:
coeffs_df = pd.DataFrame({'Country Code':list(intersect_country['Country Code'])})

In [236]:
ind_c_0 = []; ind_c_1 = []; ind_c_2 = []
com_c_0 = []; com_c_1 = []; com_c_2 = []
res_c_0 = []; res_c_1 = []; res_c_2 = []
ind_pred= []; com_pred= []; res_pred= []

#### THIS IS WHER THE FOR LOOP STARTS
for i in tqdm(range(len(intersect_country))):

    test_code = intersect_country['Country Code'].iloc[i]
    cdd_list = CDD[CDD['Country Code'] == test_code].iloc[0,2:17]
    pop_list = UN_prediction[UN_prediction['Country Code'] == test_code].iloc[0,2:17]
    ind_list = industrial[industrial['Country Code'] == test_code].iloc[0,2:17]
    com_list = commercial[commercial['Country Code'] == test_code].iloc[0,2:17]
    res_list = residential[residential['Country Code'] == test_code].iloc[0,2:17]
    dic_init = {'cdd':cdd_list, 'pop':pop_list, 'ind':ind_list, 'com':com_list, 'res':res_list}
    df_init = pd.DataFrame(dic_init)
    df_init = df_init.dropna().reset_index(drop=True)


    cdd_fut = CDD[CDD['Country Code'] == test_code].iloc[0,20:30]
    pop_fut = UN_prediction[UN_prediction['Country Code'] == test_code].iloc[0,23:33]
    dic_fut = {'cdd':cdd_fut, 'pop':pop_fut}
    df_fut = pd.DataFrame(dic_fut)
    df_fut = df_fut.dropna().reset_index(drop=True)
    
    lm_ind = linear_model.LinearRegression(normalize = True)
    X = df_init[['cdd','pop']]
    y_ind = df_init['ind']
    model = lm_ind.fit(X,y_ind)
    ind_c_0.append(lm_ind.intercept_); ind_c_1.append(lm_ind.coef_[0]); ind_c_2.append(lm_ind.coef_[1])
    ind_pred.append(lm_ind.predict(df_fut))
    
    lm_com = linear_model.LinearRegression(normalize = True)
    X = df_init[['cdd','pop']]
    y_com = df_init['com']
    model = lm_com.fit(X,y_com)
    com_c_0.append(lm_com.intercept_); com_c_1.append(lm_com.coef_[0]); com_c_2.append(lm_com.coef_[1])
    com_pred.append(lm_com.predict(df_fut))

    lm_res = linear_model.LinearRegression(normalize = True)
    X = df_init[['cdd','pop']]
    y_res = df_init['res']
    model = lm_res.fit(X,y_res)
    res_c_0.append(lm_res.intercept_); res_c_1.append(lm_res.coef_[0]); res_c_2.append(lm_res.coef_[1])
    res_pred.append(lm_res.predict(df_fut))

    
coeffs_df['inter_ind'] = ind_c_0 
coeffs_df['cdd_ind'] = ind_c_1
coeffs_df['pop_ind'] = ind_c_2
coeffs_df['inter_com'] = com_c_0
coeffs_df['cdd_com'] = com_c_1
coeffs_df['pop_com'] = com_c_2
coeffs_df['inter_res'] = res_c_0
coeffs_df['cdd_res'] = res_c_1
coeffs_df['pop_res'] = res_c_2    

100%|██████████| 99/99 [00:03<00:00, 32.42it/s]


In [252]:
industrial_pred = pd.DataFrame({'Country Code':list(intersect_country['Country Code'])})
commercial_pred = pd.DataFrame({'Country Code':list(intersect_country['Country Code'])})
residential_pred = pd.DataFrame({'Country Code':list(intersect_country['Country Code'])})
dum = [0]*len(industrial_pred)
for i in range(10):
    industrial_pred[str(2021+i)] = dum
    commercial_pred[str(2021+i)] = dum
    residential_pred[str(2021+i)]= dum
    
for i in range(len(industrial_pred)):
    industrial_pred.iloc[i,1:] = ind_pred[i]
    commercial_pred.iloc[i,1:] = com_pred[i]
    residential_pred.iloc[i,1:] = res_pred[i]

In [254]:
industrial_pred.to_csv('data/output/industrial_prediction.csv')
commercial_pred.to_csv('data/output/commercial_prediction.csv')
residential_pred.to_csv('data/output/residential_prediction.csv')

coeffs_df.to_csv('data/output/coefficients.csv')